<center><h1>Colony Initializer</h1></center>

Method to randomly initialize a colony with $n$ ants and objetive function values.

 > The food sources will be represented by the solution of the chosen Power System configuration

In [ ]:
from acopoweropt import colony, system

PSystem = system.PowerSystem(name='s10')

Colony = colony.Colony(n_ants=5, phr_evp_rate=0.25, power_system_name='s10')
Colony.initialize()

print(Colony.initial_paths)

In [ ]:
n_ants = 5

def initialize(n_ants: int):
    """Initialize colony
    
    Initializes colony by seting the ants towards random paths. Although not ideal
    each random path will make use of PowerSystem.sample_operation() and the path
    distance will be calculated using PowerSystem.solve(operation).
    
    Later improvements should aim to decouple the PowerSystem from within the colony
    initialization method.
    
    """
    
    l = []
    for ant in range(1, n_ants + 1):
        option = PSystem.sample_operation()
        result = PSystem.solve(operation=option)
        distance = trail_result.get('Ft')
        status = trail_result.get('status')

        l.append({'ant': ant, 
                  'path': option.opz.to_list(), 
                  'status': status,
                  'distance':distance,
                  'tau': 1/distance})
        
    return pd.DataFrame(l).set_index('ant') 